In [1]:
import pickle
import numpy as np
from random import shuffle
import agent_code.my_agent.algorithms as alg
from settings import s
import time
import matplotlib.pyplot as plt
import copy

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
file = open('C:/Users/Lily/Documents/UNI/FML/project/local_branch/data-collection/75-crate-no-opponents.pickle', 'rb')
data = pickle.load(file)
data = sorted(data, key=lambda k: k['state']['step'])

In [69]:
def get_blast_coords(bomb, arena, arr):
    x, y = bomb[0], bomb[1]
    if len(arr)== 0:
       arr = [(x,y)]
       #np.append(a, [[0,1]], axis=0)
    
    for i in range(1, 3+1):
        if arena[x+i,y] == -1: break
        arr.append((x+i,y))
    for i in range(1, 3+1):
        if arena[x-i,y] == -1: break
        arr.append((x-i,y))           
    for i in range(1, 3+1):
        if arena[x,y+i] == -1: break
        arr.append((x,y+i))            
    for i in range(1, 3+1):
        if arena[x,y-i] == -1: break
        arr.append((x,y-i))
    return arr

In [6]:
def look_for_targets(free_space, start, targets, logger=None):
    """Find direction of closest target that can be reached via free tiles.
    Performs a breadth-first search of the reachable free tiles until a target is encountered.
    If no target can be reached, the path that takes the agent closest to any target is chosen.
    Args:
        free_space: Boolean numpy array. True for free tiles and False for obstacles.
        start: the coordinate from which to begin the search.
        targets: list or array holding the coordinates of all target tiles.
        logger: optional logger object for debugging.
    Returns:
        coordinate of first step towards closest target or towards tile closest to any target.
    USEFUL FOR feature1, feature4
    """
    if len(targets) == 0: return None

    frontier = [start]
    parent_dict = {start: start}
    dist_so_far = {start: 0}
    best = start
    best_dist = np.sum(np.abs(np.subtract(targets, start)), axis=1).min()

    while len(frontier) > 0:
        current = frontier.pop(0)
        # Find distance from current position to all targets, track closest
        d = np.sum(np.abs(np.subtract(targets, current)), axis=1).min()
        if d + dist_so_far[current] <= best_dist:
            best = current
            best_dist = d + dist_so_far[current]
        if d == 0:
            # Found path to a target's exact position, mission accomplished!
            best = current
            break
        # Add unexplored free neighboring tiles to the queue in a random order
        x, y = current
        neighbors = [(x,y) for (x,y) in [(x+1,y), (x-1,y), (x,y+1), (x,y-1)] if free_space[x,y]]
        shuffle(neighbors)
        for neighbor in neighbors:
            if neighbor not in parent_dict:
                frontier.append(neighbor)
                parent_dict[neighbor] = current
                dist_so_far[neighbor] = dist_so_far[current] + 1
    if logger: logger.debug(f'Suitable target found at {best}')
    # Determine the first step towards the best found target tile
    current = best
    while True:
        if parent_dict[current] == start: return current
        current = parent_dict[current]

In [157]:
def feature4(game_state):
    '''
    This feature rewards the action that minimizes the distance to safety
    should the agent be in the danger zone(where explosions will be).
        F(s,a) = 1, should a reduces distance to safety
        F(s,a) = 0, otherwise
    F(s,a) returns 0 if we are not in the danger zone   
    
    We begin by extracting all relevant information from game_state
    '''
    agent = game_state['self']
    arena = game_state['arena']
    bombs = game_state['bombs']
    '''
    and initializing the resulting vector
    '''
    res = np.zeros(6, dtype=np.int8)
    if len(bombs) != 0:
        print('there are bombs')
        '''
        If there are bombs on the game board, we map all the explosions
        that will be caused by the bombs. For this, we use the help function
        get_blast_coords. This is an adjusted version of the function with the
        same name from item.py of the original framework
        '''
        danger_zone = []
        for b in bombs:
            danger_zone = get_blast_coords(b, arena, danger_zone)
        print(danger_zone)
        '''
        We then check if the agent is in the danger zone
        If agent is not in the danger zone, we return 0.
        Otherwise we calculate the distance/direction of safety.
        '''
        print(agent[0], agent[1])
        if (agent[0], agent[1]) in danger_zone:
            print('in danger zone')
            '''
            we then mark these explosions on our map. here we deep-copy
            the arena, so that in the case that the arena is needed for
            other features, it remains unchanged
            '''
            #map_ = copy.deepcopy(arena)
            #[ arena[a[0],a[1]] for a in arr]
            
            for coord in danger_zone:
                arena[coord]  = 2
            for b in bombs:
                arena[b[0], b[1]] = -1
            #map_[danger_zone[:,0], danger_zone[:,1]] = 2
            '''
            '''
            safe_loc = np.argwhere(arena==0)
            free_space = abs(arena) != 1
            print(free_space, arena)

            d = look_for_targets(free_space, (agent[0], agent[1]), safe_loc)
            print(d)
            '''
            we then calculate the minimum distance of our agent to any of these safe locations.
            For simplicity, only Manhattan distance is used to calculate distance in this
            feature extraction. However, this doesn't always represent the true distance in the 
            game because of the walls.(Possible point of improvement?)
 
            Then, we calculate the positions of our agent after taking all possible actions.
            '''
            
            actions_loc = np.array([(agent[0], agent[1]-1), #up
                                    (agent[0], agent[1]+1), #down
                                    (agent[0]-1, agent[1]), #left
                                    (agent[0]+1, agent[1]), #right
                                    (agent[0], agent[1]),   #bomb
                                    (agent[0], agent[1])])  #wait
            
            res = (actions_loc[:,0] == d[0]) & (actions_loc[:,1] == d[1])
            res = res.astype(int)
     
    return res
    

In [158]:
# to easily change to different game states
i = 2
#extract information from game_state
game_state = data[i]['state']
explosions_i = np.argwhere(game_state['explosions'] != 0)
print(explosions_i)

[]


In [159]:
#feat_dist_to_safe(game_state)
#print(feature4(game_state), data[i+1]['next-action'])
#print(game_state['self'], data[i-1]['state']['self'])
#feature4(game_state)
feature4(game_state)

there are bombs
[(14, 1), (15, 1), (13, 1), (12, 1), (11, 1)]
15 1
in danger zone
[[False False False False False False False False False False False False
  False False False False False]
 [False  True  True False False False False False False  True False False
   True False  True  True False]
 [False  True False False False  True False False False  True False False
  False False False  True False]
 [False False False False False False  True False False False False False
  False False False  True False]
 [False False False False False False False False False False False  True
  False False False False False]
 [False  True False  True False False False False False False False False
   True  True  True False False]
 [False False False False False  True False False False False False False
  False False False False False]
 [False  True False False False False False  True False False False False
   True False False False False]
 [False  True False  True False False False False False False 

array([0, 1, 0, 0, 0, 0])

In [111]:
arena[explosions_i[:,0], explosions_i[:,1]]
game_state['bombs']

[(15, 2, 0)]

In [105]:
arr = []
b = game_state['bombs'][0]
arr = get_blast_coords(b, game_state['arena'], arr)
#arr.append((2,3))
arena = game_state['arena']
print(arr)
for a in arr:
    arena[a] = 2
np.ar

[(15, 2), (15, 3), (15, 4), (15, 5), (15, 1)]


array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1],
       [-1,  0,  0,  1,  1,  1,  1,  1,  1,  0,  1,  1,  0,  1,  0,  0,
        -1],
       [-1,  0, -1,  1, -1,  0, -1,  1, -1,  0, -1,  1, -1,  1, -1,  0,
        -1],
       [-1,  1,  1,  1,  1,  1,  0,  1,  1,  1,  1,  1,  1,  1,  1,  0,
        -1],
       [-1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  0, -1,  1, -1,  1,
        -1],
       [-1,  0,  1,  0,  1,  1,  1,  1,  1,  1,  1,  1,  0,  0,  0,  1,
        -1],
       [-1,  1, -1,  1, -1,  0, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,
        -1],
       [-1,  0,  1,  1,  1,  1,  1,  0,  1,  1,  1,  1,  0,  1,  1,  1,
        -1],
       [-1,  0, -1,  0, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,
        -1],
       [-1,  1,  1,  1,  0,  0,  0,  0,  1,  0,  1,  0,  1,  1,  1,  1,
        -1],
       [-1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  0,
        -1],
       [-1,  0,  1,  1,  1,  1,  1,  1,  1,  0,  1,  1,  1,  0,  

In [156]:
a = np.arange(4)
print(a)
print(np.max(a))

[0 1 2 3]
3
